## Use case

Tagging은 다음과 같은 클래스로 문서에 라벨을 붙이는 것을 의미합니다::

- sentiment 감정
- language 언어
- style (formal, informal etc.) 언어의 스타일
- covered topics 다루는 주제들
- political tendency 정치 경향성

## Overview

Tagging has a few components:

* `function`: 추출과 마찬가지로, 태깅은 모델이 문서를 태깅하는 방법을 지정하는 함수를 사용합니다.
* `schema`: 문서를 태깅하는 방법을 정의합니다.

## Quickstart

LangChain에서 OpenAI 함수를 사용하여 태깅을 어떻게 할 수 있는지에 대한 매우 간단한 예를 살펴보겠습니다.

In [ ]:
!pip install langchain openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.5/257.5 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 1.5 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24

In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = #여기에 GPT API Key를 발급 받아서 넣어야함

In [ ]:
from langchain.chains import create_tagging_chain, create_tagging_chain_pydantic
from langchain.chat_models import ChatOpenAI

우리는 스키마에서 예상되는 유형과 함께 몇 가지 속성을 지정합니다.

In [ ]:
# Schema
schema = {
    "properties": {
        "sentiment": {"type": "string"},
        "aggressiveness": {"type": "integer"},
        "language": {"type": "string"},
    }
}

# LLM
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
chain = create_tagging_chain(schema, llm)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [ ]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
chain.run(inp)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'sentiment': 'positive', 'language': 'Spanish'}

In [ ]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
chain.run(inp)

{'sentiment': 'enojado', 'aggressiveness': 1, 'language': 'es'}

예제에서 볼 수 있듯이, 우리가 원하는 것을 정확하게 해석합니다.

예를 들어 다른 언어로 된 감정 표현을 얻을 수 있도록 결과가 다양합니다.

다음 섹션에서 이러한 결과를 제어하는 방법을 살펴보겠습니다.

## Finer control

스키마를 신중하게 정의하면 모델의 출력을 더 세밀하게 제어할 수 있습니다.

구체적으로, 각 속성의

- 각 속성에 대해 가능한 값
- 모델이 속성을 이해하도록 하기 위한 설명
- 반환해야 하는 필수 속성

다음은 앞서 언급한 각 측면을 제어하기 위해 `_enum_`, `_description_` 및 `_required_`를 사용하는 방법의 예시입니다:

In [ ]:
schema = {
    "properties": {
        "aggressiveness": {
            "type": "integer",
            "enum": [1, 2, 3, 4, 5],
            "description": "describes how aggressive the statement is, the higher the number the more aggressive",
        },
        "language": {
            "type": "string",
            "enum": ["spanish", "english", "french", "german", "italian"],
        },
    },
    "required": ["language", "aggressiveness"],
}

In [ ]:
chain = create_tagging_chain(schema, llm)

이제 답변이 훨씬 더 좋아졌습니다!

In [ ]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
chain.run(inp)

{'aggressiveness': '3', 'language': 'spanish'}

In [ ]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
chain.run(inp)

{'aggressiveness': '5', 'language': 'spanish'}

In [ ]:
inp = "Weather is ok here, I can go outside without much more than a coat"
chain.run(inp)

{'aggressiveness': '3', 'language': 'english'}

## Pydantic

또한 Pydantic 스키마를 사용하여 필요한 속성 및 유형을 지정할 수도 있습니다.

열거형` 또는 `설명`과 같은 다른 인수를 각 필드에 보낼 수도 있습니다.

이렇게 하면 파이썬에서 순전히 파이썬 타입으로 새 클래스나 함수를 만들 때와 같은 방식으로 스키마를 지정할 수 있습니다.

In [ ]:
from langchain.pydantic_v1 import BaseModel, Field

In [ ]:
class Tags(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    aggressiveness: int = Field(
        ...,
        description="describes how aggressive the statement is, the higher the number the more aggressive",
        enum=[1, 2, 3, 4, 5],
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )

In [ ]:
chain = create_tagging_chain_pydantic(Tags, llm)

In [ ]:
inp = "Estoy muy enojado con vos! Te voy a dar tu merecido!"
res = chain.run(inp)

In [ ]:
res

Tags(sentiment='sad', aggressiveness=5, language='spanish')

## 연습문제
### 1. 스키마 정의
* 특정 태깅 요구에 맞게 스키마를 맞춤화하는 방법을 이해합니다.


```
1. 스키마 개념 이해: 스키마가 태깅 시스템에서 어떻게 사용되는지 이해합니다. 스키마는 문서의 특정 속성을 정의하며, 이를 통해 모델이 어떤 정보를 추출하고 태깅할지 결정합니다.

2. 속성 선택: 유머, 격식, 주제 관련성 등의 속성을 선택합니다. 각 속성이 문서에 어떻게 나타날 수 있는지 생각해 봅니다.

3. 스키마 정의: 선택한 속성을 바탕으로 JSON 형식의 스키마를 작성합니다.
예를 들어, humor는 "none", "some", "a lot"과 같이, 격식은 "informal", "neutral", "formal"과 같이, 주제 관련성은 주제 목록을 포함하도록 할 수 있습니다.
```



In [ ]:
# 새로운 스키마 정의
schema = {
    "properties": {
        "humor": {"type": "string", "enum": ["none", "some", "a lot"]},
        "formality": {"type": "string", "enum": ["informal", "neutral", "formal"]},
    }
}

# 언어 모델 생성
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

# 태깅 체인 생성
chain = create_tagging_chain(schema, llm)

# 태깅을 위한 입력 문서
inputs = [
    "This is a very formal document discussing scientific research.",
    "Hey there! Just chilling and having fun with AI."
]

# 각 입력에 대해 태깅 실행
for inp in inputs:
    result = chain.run(inp)
    print(f"Input: {inp}\nTagged Output: {result}\n")

Input: This is a very formal document discussing scientific research.
Tagged Output: {'formality': 'formal'}

Input: Hey there! Just chilling and having fun with AI.
Tagged Output: {'humor': 'some', 'formality': 'informal'}



In [ ]:
#pydantic
class Tags(BaseModel):
    humor: str = Field(..., enum=["none", "some", "a lot"])
    formality: str = Field(..., enum=["informal", "neutral", "formal"])

# 언어 모델 생성
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")

# 태깅 체인 생성
chain = create_tagging_chain_pydantic(Tags, llm)

# 태깅을 위한 입력 문서
inputs = [
    "This is a very formal document discussing scientific research.",
    "Hey there! Just chilling and having fun with AI."
]

# 각 입력에 대해 태깅 실행
for inp in inputs:
    result = chain.run(inp)
    print(f"Input: {inp}\nTagged Output: {result}\n")

Input: This is a very formal document discussing scientific research.
Tagged Output: humor='none' formality='formal'

Input: Hey there! Just chilling and having fun with AI.
Tagged Output: humor='some' formality='informal'



### 2. 입력처리
* 다양한 입력에 대해 태깅 모델의 출력을 처리하고 해석하는 연습을 합니다.
* 다양한 언어와 스타일로 된 다양한 입력을 생성하고, 이를 태깅 체인을 사용하여 태그를 붙입니다.



```
1. 다양한 입력 준비: 서로 다른 언어, 스타일, 주제를 가진 여러 텍스트 문서를 준비합니다.
예를 들어, 감성적인 일기, 공식적인 비즈니스 보고서, 비공식적인 대화 등을 포함할 수 있습니다.

2. 태깅 체인 설정: 이미 만들어진 태깅 체인을 사용하거나, 필요에 따라 새로운 스키마를 정의하여 태깅 체인을 설정합니다.
    * 감정: happy, neutral, sad
    * 언어: English, Korean, Japanese, Franch, Spainish
    * 정중함: informal, neutral, formal

3. 태깅 실행: 준비한 각 입력에 대해 태깅 체인을 실행합니다. 이때 각 입력에 대해 태그된 결과를 기록합니다.

4. 결과 분석: 태깅 결과를 분석하여 모델이 어떻게 다양한 스타일과 언어의 입력을 처리하는지 관찰합니다. 모델이 잘 처리하는 부분과 개선이 필요한 부분을 식별합니다.
```



In [ ]:
# 태깅을 위한 스키마 (기존에 정의된 스키마 사용)
schema = {
    "properties": {
        "sentiment": {"type": "string", "enum": ["happy", "neutral", "sad"]},
        "language": {"type": "string",
                     "enum": ["English", "Korean", "Japanese", "Franch", "Spainish"]},
        "formality": {"type": "string", "enum": ["informal", "neutral", "formal"]}
    }
}

# 언어 모델 생성
llm = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo-0613")

# 태깅 체인 생성
chain = create_tagging_chain(schema, llm)

# 다양한 언어와 스타일의 입력
inputs = [
    "오늘은 정말 기분이 좋아. 친구들과 함께 즐거운 시간을 보냈어.",
    "This is an official document regarding the financial report of the last quarter.",
    "Salut! Comment ça va? Je suis très excité pour le match ce soir!",
    "Gestern war ein anstrengender Tag, aber ich habe viel erreicht.",
    "¿Cómo se podría mejorar el rendimiento de nuestro equipo?",
    "忙しい一日だったけど、とても充実していたよ。"
]

# 각 입력에 대해 태깅 실행
for inp in inputs:
    result = chain.run(inp)
    print(f"Input: {inp}\nTagged Output: {result}\n")

Input: 오늘은 정말 기분이 좋아. 친구들과 함께 즐거운 시간을 보냈어.
Tagged Output: {'sentiment': 'happy', 'language': 'Korean'}

Input: This is an official document regarding the financial report of the last quarter.
Tagged Output: {}

Input: Salut! Comment ça va? Je suis très excité pour le match ce soir!
Tagged Output: {'sentiment': 'neutral', 'formality': 'informal'}

Input: Gestern war ein anstrengender Tag, aber ich habe viel erreicht.
Tagged Output: {'sentiment': 'neutral', 'language': 'German'}

Input: ¿Cómo se podría mejorar el rendimiento de nuestro equipo?
Tagged Output: {'language': 'Spanish'}

Input: 忙しい一日だったけど、とても充実していたよ。
Tagged Output: {'sentiment': 'neutral', 'language': 'Japanese', 'formality': 'informal'}



In [ ]:
# dydantic class
class Tags(BaseModel):
    sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
    style: str = Field(..., enum=["English", "Korean", "Japanese", "Franch", "Spainish"])
    language: str = Field(..., enum=["informal", "neutral", "formal"])


# 언어 모델 생성
llm = ChatOpenAI(temperature=0.7, model="gpt-3.5-turbo-0613")

# 태깅 체인 생성
chain = create_tagging_chain_pydantic(Tags, llm)

# 다양한 언어와 스타일의 입력
inputs = [
    "오늘은 정말 기분이 좋아. 친구들과 함께 즐거운 시간을 보냈어.",
    "This is an official document regarding the financial report of the last quarter.",
    "Salut! Comment ça va? Je suis très excité pour le match ce soir!",
    "Gestern war ein anstrengender Tag, aber ich habe viel erreicht.",
    "¿Cómo se podría mejorar el rendimiento de nuestro equipo?",
    "忙しい一日だったけど、とても充実していたよ。"
]

# 각 입력에 대해 태깅 실행
for inp in inputs:
    result = chain.run(inp)
    print(f"Input: {inp}\nTagged Output: {result}\n")

Input: 오늘은 정말 기분이 좋아. 친구들과 함께 즐거운 시간을 보냈어.
Tagged Output: sentiment='happy' style='Korean' language='informal'

Input: This is an official document regarding the financial report of the last quarter.
Tagged Output: sentiment='neutral' style='English' language='formal'

Input: Salut! Comment ça va? Je suis très excité pour le match ce soir!
Tagged Output: sentiment='neutral' style='Franch' language='informal'

Input: Gestern war ein anstrengender Tag, aber ich habe viel erreicht.
Tagged Output: sentiment='neutral' style='German' language='neutral'

Input: ¿Cómo se podría mejorar el rendimiento de nuestro equipo?
Tagged Output: sentiment='neutral' style='Spanish' language='neutral'

Input: 忙しい一日だったけど、とても充実していたよ。
Tagged Output: sentiment='neutral' style='Japanese' language='informal'

